In [254]:
%matplotlib inline
from importlib import reload
import utils; reload(utils)
from utils import *
from __future__ import division, print_function

In [255]:
model_path = 'data/imdb/models/'

## Setup data

We're going to look at the IMDB dataset, which contains movie reviews from IMDB, along with their sentiment. Keras comes with some helpers for this dataset.

In [256]:
from keras.datasets import imdb
idx = imdb.get_word_index()

This is the word list:

In [257]:
idx_arr = sorted(idx, key=idx.get)
idx_arr[:10]

['the', 'and', 'a', 'of', 'to', 'is', 'br', 'in', 'it', 'i']

...and this is the mapping from id to word

In [258]:
idx2word = {v: k for k, v in idx.items()}

We download the reviews using code copied from keras.datasets:

In [259]:
path = get_file('imdb_full.pkl',
                origin='https://s3.amazonaws.com/text-datasets/imdb_full.pkl',
                md5_hash='d091312047c43cf9e4e38fef92437263')
f = open(path, 'rb')
(x_train, labels_train), (x_test, labels_test) = pickle.load(f)

In [260]:
len(x_train), len(x_test)

(25000, 25000)

Here's the 1st review. As you see, the words have been replaced by ids. The ids can be looked up in idx2word.

In [261]:
', '.join(map(str, x_train[0]))

'23022, 309, 6, 3, 1069, 209, 9, 2175, 30, 1, 169, 55, 14, 46, 82, 5869, 41, 393, 110, 138, 14, 5359, 58, 4477, 150, 8, 1, 5032, 5948, 482, 69, 5, 261, 12, 23022, 73935, 2003, 6, 73, 2436, 5, 632, 71, 6, 5359, 1, 25279, 5, 2004, 10471, 1, 5941, 1534, 34, 67, 64, 205, 140, 65, 1232, 63526, 21145, 1, 49265, 4, 1, 223, 901, 29, 3024, 69, 4, 1, 5863, 10, 694, 2, 65, 1534, 51, 10, 216, 1, 387, 8, 60, 3, 1472, 3724, 802, 5, 3521, 177, 1, 393, 10, 1238, 14030, 30, 309, 3, 353, 344, 2989, 143, 130, 5, 7804, 28, 4, 126, 5359, 1472, 2375, 5, 23022, 309, 10, 532, 12, 108, 1470, 4, 58, 556, 101, 12, 23022, 309, 6, 227, 4187, 48, 3, 2237, 12, 9, 215'

The first word of the first review is 23022. Let's see what that is.

In [262]:
idx2word[23022]

'bromwell'

Here's the whole review, mapped from ids to words.

In [263]:
' '.join([idx2word[o] for o in x_train[0]])

"bromwell high is a cartoon comedy it ran at the same time as some other programs about school life such as teachers my 35 years in the teaching profession lead me to believe that bromwell high's satire is much closer to reality than is teachers the scramble to survive financially the insightful students who can see right through their pathetic teachers' pomp the pettiness of the whole situation all remind me of the schools i knew and their students when i saw the episode in which a student repeatedly tried to burn down the school i immediately recalled at high a classic line inspector i'm here to sack one of your teachers student welcome to bromwell high i expect that many adults of my age think that bromwell high is far fetched what a pity that it isn't"

The labels are 1 for positive, 0 for negative.

In [20]:
labels_train[:10]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

Reduce vocab size by setting rare words to max index.

In [264]:
vocab_size = 5000

trn = [np.array([i if i<vocab_size-1 else vocab_size-1 for i in s]) for s in x_train]
test = [np.array([i if i<vocab_size-1 else vocab_size-1 for i in s]) for s in x_test]

Look at distribution of lengths of sentences.

In [242]:
[len(item) for item in trn]
np.array(map(len, trn))

array(<map object at 0x7fc861468dd8>, dtype=object)

In [265]:
lens = np.array([len(item) for item in trn])
(lens.max(), lens.min(), lens.mean())

(2493, 10, 237.71364)

Pad (with zero) or truncate each sentence to make consistent length.

In [266]:
seq_len = 500

trn = sequence.pad_sequences(trn, maxlen=seq_len, value=0)
test = sequence.pad_sequences(test, maxlen=seq_len, value=0)

This results in nice rectangular matrices that can be passed to ML algorithms. Reviews shorter than 500 words are pre-padded with zeros, those greater are truncated.

In [39]:
trn.shape

(25000, 500)

## Create simple models

### Single hidden layer NN

The simplest model that tends to give reasonable results is a single hidden layer net. So let's try that. Note that we can't expect to get any useful results by feeding word ids directly into a neural net - so instead we use an embedding to replace them with a vector of 32 (initially random) floats for each word in the vocab.

In [147]:
model = Sequential([
    Embedding(vocab_size, 50, input_length=seq_len),
    Flatten(),
    Dense(100, activation='relu'),
    Dropout(0.7),
    Dense(1, activation='sigmoid')])

In [148]:
model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_24 (Embedding)     (None, 500, 50)           250000    
_________________________________________________________________
flatten_22 (Flatten)         (None, 25000)             0         
_________________________________________________________________
dense_43 (Dense)             (None, 100)               2500100   
_________________________________________________________________
dropout_60 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_44 (Dense)             (None, 1)                 101       
Total params: 2,750,201
Trainable params: 2,750,201
Non-trainable params: 0
_________________________________________________________________


In [149]:
model.fit(trn, labels_train, validation_data=(test, labels_test), epochs=2, batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 6s - loss: 0.4702 - acc: 0.7421 - val_loss: 0.3109 - val_acc: 0.8681
Epoch 2/2
25000/25000 [==============================] - 5s - loss: 0.1986 - acc: 0.9251 - val_loss: 0.3048 - val_acc: 0.8749


The [stanford paper](http://ai.stanford.edu/~amaas/papers/wvSent_acl2011.pdf) that this dataset is from cites a state of the art accuracy (without unlabelled data) of 0.883. So we're short of that, but on the right track.

### Single conv layer with max pooling

A CNN is likely to work better, since it's designed to take advantage of ordered data. We'll need to use a 1D CNN, since a sequence of words is 1D.

In [76]:
from keras.layers import SpatialDropout1D

In [150]:
conv1 = Sequential([
    Embedding(vocab_size, 50, input_length=seq_len),
    SpatialDropout1D(0.2),
#     Dropout(0.2),
    Conv1D(64, 5, padding='same', activation='relu'),
    Dropout(0.2),
    MaxPooling1D(),
    Flatten(),
    Dense(100, activation='relu'),
    Dropout(0.7),
    Dense(1, activation='sigmoid')])

In [151]:
conv1.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [152]:
conv1.fit(trn, labels_train, validation_data=(test, labels_test), epochs=4, batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/4
25000/25000 [==============================] - 9s - loss: 0.4418 - acc: 0.7644 - val_loss: 0.2654 - val_acc: 0.8904
Epoch 2/4
25000/25000 [==============================] - 9s - loss: 0.2394 - acc: 0.9124 - val_loss: 0.2537 - val_acc: 0.8947
Epoch 3/4
25000/25000 [==============================] - 9s - loss: 0.1939 - acc: 0.9306 - val_loss: 0.2643 - val_acc: 0.8926
Epoch 4/4
25000/25000 [==============================] - 9s - loss: 0.1596 - acc: 0.9431 - val_loss: 0.2887 - val_acc: 0.8916


That's well past the Stanford paper's accuracy - another win for CNNs!

In [87]:
conv1.save_weights(model_path + 'conv1.h5')

In [88]:
conv1.load_weights(model_path + 'conv1.h5')

## Pre-trained vectors

You may want to look at wordvectors.ipynb before moving on.

In this section, we replicate the previous CNN, but using pre-trained embeddings.

In [267]:
def load_vectors(loc):
    return (load_array(loc+'.dat'),
        pickle.load(open(loc+'_words.pkl','rb'), encoding = 'utf-8'),
        pickle.load(open(loc+'_idx.pkl','rb'), encoding = 'utf-8'))

In [268]:
vecs, words, wordidx = load_vectors('data/glove/results/6B.50d')

The glove word ids and imdb word ids use different indexes. So we create a simple function that creates an embedding matrix using the indexes from imdb, and the embeddings from glove (where they exist).

In [269]:
hasword = 0

In [270]:
def create_emb():
    global hasword
    n_fact = vecs.shape[1]
    emb = np.zeros((vocab_size, n_fact))

    for i in range(1,len(emb)):
        word = idx2word[i]
        if word and re.match(r"^[a-zA-Z0-9\-]*$", word):
            hasword += 1
            src_idx = wordidx[word]
            emb[i] = vecs[src_idx]
        else:
            # If we can't find the word in glove, randomly initialize
            emb[i] = normal(scale=0.6, size=(n_fact,))

    # This is our "rare word" id - we want to randomly initialize
    emb[-1] = normal(scale=0.6, size=(n_fact,))
    emb/=3
    return emb

In [ ]:
len(vecs), len(words), len(wordidx)

In [271]:
emb = create_emb()

In [272]:
hasword, len(emb)

(4914, 5000)

We pass our embedding matrix to the Embedding constructor, and set it to non-trainable.

In [273]:
rmsprop = keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
adam = Adam()

In [274]:
model = Sequential([
    Embedding(vocab_size, 50, input_length=seq_len, 
              weights=[emb], trainable=False),
    SpatialDropout1D(0.2),
#     Dropout(0.25),
    Conv1D(64, 5, padding='same', activation='relu'),
#     Dropout(0.2),
    MaxPooling1D(),
    Flatten(),
    Dense(100, activation='relu'),
    Dropout(0.7),
    Dense(1, activation='sigmoid')])

In [275]:
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

In [276]:
model.fit(trn, labels_train, validation_data=(test, labels_test), epochs=5, batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/5
25000/25000 [==============================] - 8s - loss: 0.6044 - acc: 0.6694 - val_loss: 0.4975 - val_acc: 0.7701
Epoch 2/5
25000/25000 [==============================] - 6s - loss: 0.4987 - acc: 0.7637 - val_loss: 0.4265 - val_acc: 0.8156
Epoch 3/5
25000/25000 [==============================] - 6s - loss: 0.4442 - acc: 0.7988 - val_loss: 0.4158 - val_acc: 0.8091
Epoch 4/5
25000/25000 [==============================] - 6s - loss: 0.4013 - acc: 0.8188 - val_loss: 0.3794 - val_acc: 0.8336
Epoch 5/5
25000/25000 [==============================] - 6s - loss: 0.3732 - acc: 0.8350 - val_loss: 0.3764 - val_acc: 0.8336


We already have beaten our previous model! But let's fine-tune the embedding weights - especially since the words we couldn't find in glove just have random embeddings.

In [277]:
model.layers[0].trainable=True

In [ ]:
model.optimizer.lr=1e-4

In [ ]:
model.fit(trn, labels_train, validation_data=(test, labels_test), epochs=5, batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/5
25000/25000 [==============================] - 7s - loss: 0.3514 - acc: 0.8483 - val_loss: 0.3723 - val_acc: 0.8330
Epoch 2/5
25000/25000 [==============================] - 6s - loss: 0.3282 - acc: 0.8587 - val_loss: 0.3972 - val_acc: 0.8210
Epoch 3/5
25000/25000 [==============================] - 6s - loss: 0.3049 - acc: 0.8708 - val_loss: 0.3931 - val_acc: 0.8262
Epoch 4/5
25000/25000 [==============================] - 6s - loss: 0.2830 - acc: 0.8811 - val_loss: 0.4505 - val_acc: 0.8074
Epoch 5/5
 6720/25000 [=======>......................] - ETA: 3s - loss: 0.2556 - acc: 0.8912

As expected, that's given us a nice little boost. :)

In [282]:
model.save_weights(model_path+'glove50.h5')

## Multi-size CNN

This is an implementation of a multi-size CNN as shown in Ben Bowles' [excellent blog post](https://quid.com/feed/how-quid-uses-deep-learning-with-small-data).

In [283]:
from keras.layers.merge import concatenate

We use the functional API to create multiple conv layers of different sizes, and then concatenate them.

In [284]:
graph_in = Input ((vocab_size, 50))
convs = [ ] 
for fsz in range (3, 6): 
    x = Conv1D(64, fsz, padding='same', activation="relu")(graph_in)
    x = MaxPooling1D()(x) 
    x = Flatten()(x) 
    convs.append(x)
out = concatenate(convs) 
graph = Model(graph_in, out) 

In [285]:
emb = create_emb()

We then replace the conv/max-pool layer in our original CNN with the concatenated conv layers.

In [286]:
model = Sequential ([
    Embedding(vocab_size, 50, input_length=seq_len, weights=[emb]),
    SpatialDropout1D(0.2),
    Dropout (0.2),
    graph,
    Dropout (0.5),
    Dense (100, activation="relu"),
    Dropout (0.7),
    Dense (1, activation='sigmoid')
    ])

In [287]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_47 (Embedding)     (None, 500, 50)           250000    
_________________________________________________________________
spatial_dropout1d_28 (Spatia (None, 500, 50)           0         
_________________________________________________________________
dropout_103 (Dropout)        (None, 500, 50)           0         
_________________________________________________________________
model_6 (Model)              multiple                  38592     
_________________________________________________________________
dropout_104 (Dropout)        (None, 48000)             0         
_________________________________________________________________
dense_89 (Dense)             (None, 100)               4800100   
_________________________________________________________________
dropout_105 (Dropout)        (None, 100)               0         
__________

In [288]:
model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [289]:
model.fit(trn, labels_train, validation_data=(test, labels_test), epochs=2, batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 22s - loss: 0.4761 - acc: 0.7575 - val_loss: 0.2823 - val_acc: 0.8835
Epoch 2/2
25000/25000 [==============================] - 20s - loss: 0.2985 - acc: 0.8819 - val_loss: 0.2637 - val_acc: 0.8925


Interestingly, I found that in this case I got best results when I started the embedding layer as being trainable, and then set it to non-trainable after a couple of epochs. I have no idea why!

In [290]:
model.layers[0].trainable=False

In [291]:
model.optimizer.lr=1e-5

In [292]:
model.fit(trn, labels_train, validation_data=(test, labels_test), epochs=2, batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 20s - loss: 0.2453 - acc: 0.9057 - val_loss: 0.2504 - val_acc: 0.8974
Epoch 2/2
25000/25000 [==============================] - 20s - loss: 0.2222 - acc: 0.9154 - val_loss: 0.2446 - val_acc: 0.9001


This more complex architecture has given us another boost in accuracy.

## LSTM

We haven't covered this bit yet!

In [ ]:
model = Sequential([
    Embedding(vocab_size, 32, input_length=seq_len, mask_zero=True,
              embeddings_regularizer=l2(1e-6)),
    SpatialDropout1D(0.2),
    LSTM(100, implementation=2),
    Dense(1, activation='sigmoid')])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_54 (Embedding)     (None, 500, 32)           160000    
_________________________________________________________________
spatial_dropout1d_34 (Spatia (None, 500, 32)           0         
_________________________________________________________________
lstm_7 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_97 (Dense)             (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(trn, labels_train, validation_data=(test, labels_test), epochs=1, batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/1
 7744/25000 [========>.....................] - ETA: 397s - loss: 0.6204 - acc: 0.6600